## Import Libraries

In [1]:
from time import sleep

In [2]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [3]:
import matplotlib.pyplot as plt

In [4]:
from pydgilib_extra import *

Set the path to `dgilib.dll`.

In [5]:
dgilib_path = "C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Python\\dgilib.dll"

## Data Logging

Create a figure for the plot.

In [6]:
fig = plt.figure(figsize=(10, 6))

Create the configuration dictionary for `DGILibExtra`.

In [7]:
config_dict = {
    "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
    "read_mode": [True, True, True, True],
    "write_mode": [False, False, False, False],
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "augment_gpio" : True,
    "fig": fig,
    "verbose": 0,
}

Perform the measurement.

In [8]:
data = []
fig.clf()

with DGILibExtra(dgilib_path, **config_dict) as dgilib:
    dgilib.device_reset()
    dgilib.logger_start()
    sleep(1)
    dgilib.update_callback()
    while not all(dgilib.data[INTERFACE_GPIO][1][-1]):
        dgilib.update_callback()
#         sleep(1)
    sleep(0.1)
    dgilib.logger_stop()
    data = dgilib.data

## Analysis

In [9]:
encrypt_charge, encrypt_time = power_and_time_per_pulse(data, 2)

In [10]:
decrypt_charge, decrypt_time = power_and_time_per_pulse(data, 3)

In [11]:
MBEDTLS_AES_BLOCK_SIZE = 16
MIN_AES_BLOCKS = 1
num_bytes = range(MIN_AES_BLOCKS * MBEDTLS_AES_BLOCK_SIZE, MBEDTLS_AES_BLOCK_SIZE * (MIN_AES_BLOCKS + len(encrypt_charge)), MBEDTLS_AES_BLOCK_SIZE)

In [12]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [13]:
results = []
for y in [encrypt_charge, decrypt_charge, encrypt_time, decrypt_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 400
    # variables        = 2
    chi-square         = 1.3250e-06
    reduced chi-square = 3.3291e-09
    Akaike info crit   = -7806.23289
    Bayesian info crit = -7798.24996
[[Variables]]
    slope:      3.2317e-05 +/- 1.5615e-09 (0.00%) (init = 0)
    intercept: -2.2451e-04 +/- 5.7807e-06 (2.57%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.867

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 400
    # variables        = 2
    chi-square         = 1.4053e-06
    reduced chi-square = 3.5309e-09
    Akaike info crit   = -7782.69342
    Bayesian info crit = -7774.71049
[[Variables]]
    slope:      3.3292e-05 +/- 1.6081e-09 (0.00%) (init = 0)
    intercept: -2.1837e-04 +/- 5.9533e-06 (2.73%) (init = 1)
[[Correlations]] (unrepor

In [14]:
fig = plt.figure(figsize=(9, 6))

In [15]:
charge_color = 'r'
time_color = 'b'

In [16]:
fig.clf()
# fig.suptitle("Energy analysis of AES")
ax1 = fig.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Total Charge [mC]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [17]:
lines = []
lines += ax1.plot(num_bytes, encrypt_charge, charge_color+'-', label='Encrypt Charge')
lines += ax1.plot(num_bytes, decrypt_charge, charge_color+'--', label='Decrypt Charge')
lines += ax2.plot(num_bytes, encrypt_time, time_color+'-', label='Encrypt Time')
lines += ax2.plot(num_bytes, decrypt_time, time_color+'--', label='Decrypt Time')
ax1.legend(handles=lines)
ax1.set_title(f"Encrypt Charge: Base {results[0].params['intercept'].value:.03} mC plus {results[0].params['slope'].value:.03} mC per byte\n" +
             f"Decrypt Charge: Base {results[1].params['intercept'].value:.03} mC plus {results[1].params['slope'].value:.03} mC per byte\n" +
             f"Encrypt Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"Decrypt Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n")
fig.tight_layout()
fig.show()

In [ ]:
# data